In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### 1 Data Processing

In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_set = train_datagen.flow_from_directory(
    '/content/gdrive/My Drive/dataset_train',
    target_size=(64, 64),
    batch_size=32,
    classes=['category 1', 'category 2', 'category 3', 'category 4'],
    class_mode='categorical'
)


Found 88 images belonging to 4 classes.


In [ ]:
train_set.image_shape

The <b>image shape</b> of each training observation is <b>(64, 64, 3)</b>
An image shape of 64 x 64 x 3 means that the image is a square with a height and width of 64 pixels, and has 3 color channels: red, green, and blue (RGB). 

In [ ]:
set(train_set.classes)

Our train set has <b>4 classes</b> to predict on.

### 2 Initial Classifier Build

In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Create the classifier instance
classifier = Sequential()

# Add the first convolutional layer
classifier.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=(64, 64, 3), activation='relu'))

# Add the first pooling layer
classifier.add(MaxPooling2D(pool_size=(2,2)))

# Add the second convolutional layer
classifier.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))

# Add the second pooling layer
classifier.add(MaxPooling2D(pool_size=(2,2)))

# Flatten the output from the convolutional layers
classifier.add(Flatten())

# Add the first fully connected layer
classifier.add(Dense(units=128, activation='relu'))

# Add the output layer
classifier.add(Dense(units=4, activation='softmax'))

# Compile the classifier
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


### 3 Model Runs

In [6]:
# Train the model
classifier.fit(
    train_set,
    steps_per_epoch=3,
    epochs=3
)

# Save the model to your Google Drive
classifier.save('/content/gdrive/My Drive/model/img_class.h5')
print("Saved model")


Epoch 1/3
3/3 [==============================] - 21s 4s/step - loss: 1.3622 - accuracy: 0.3523
Epoch 2/3
3/3 [==============================] - 1s 373ms/step - loss: 0.6888 - accuracy: 0.8068
Epoch 3/3
3/3 [==============================] - 2s 685ms/step - loss: 0.3797 - accuracy: 0.8977
Saved model


In [7]:
import os, glob
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('/content/gdrive/My Drive/model/img_class.h5')
print("Loaded model from disk")

# Path to test data
img_dir = "/content/gdrive/My Drive/dataset_test"

# Iterate over each test image
data_path = os.path.join(img_dir, '*g')
files = glob.glob(data_path)

# Print the files in the dataset_test folder
for f in files:
    print(f)

# Make a prediction and add to results
data = []
results = []
for f1 in files:
    img = image.load_img(f1, target_size=(64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    data.append(img)
    result = model.predict(img)
    r = np.argmax(result, axis=1)
    results.append(r)

print(results)

Loaded model from disk
/content/gdrive/My Drive/dataset_test/4011.png
/content/gdrive/My Drive/dataset_test/1022.png
/content/gdrive/My Drive/dataset_test/6023.png
/content/gdrive/My Drive/dataset_test/C014.png
/content/gdrive/My Drive/dataset_test/C033.png
/content/gdrive/My Drive/dataset_test/1053.png
/content/gdrive/My Drive/dataset_test/4053.png
/content/gdrive/My Drive/dataset_test/6051.png
1/1 [==============================] - 0s 21ms/step
[array([0]), array([0]), array([1]), array([3]), array([3]), array([0]), array([2]), array([1])]


In [9]:
# Check category labels in training_set
train_set.class_indices

{'category 1': 0, 'category 2': 1, 'category 3': 2, 'category 4': 3}

In [10]:
# Manually create the test labels 
test_label = [2, 0, 1, 3, 3, 0, 2, 1]

In [11]:
from sklearn.metrics import accuracy_score

# Convert results to a 1D numpy array
y_pred_test = np.array([pred[0] for pred in results])

# Compute the accuracy score
accuracy = accuracy_score(test_label, y_pred_test)

# Print the accuracy score
print(f'Accuracy: {accuracy*100}%')

Accuracy: 87.5%


In [13]:
import itertools
# Define a list to store the results
model_results = []

# Define a list of tuples for the different combinations of steps_per_epoch and epochs
combos = [(1, 1), (1, 2), (1, 3), (2, 4), (2, 5), (2, 6), (3, 7), (3, 8), (5, 9), (5, 10)]

# Loop through each combination and train the model
for combo in combos:
    steps_per_epoch = combo[0]
    epochs = combo[1]
    print(f'Training model with steps_per_epoch={steps_per_epoch} and epochs={epochs}')

    train_set = train_datagen.flow_from_directory(
    '/content/gdrive/My Drive/dataset_train',
    target_size=(64, 64),
    batch_size=8,
    classes=['category 1', 'category 2', 'category 3', 'category 4'],
    class_mode='categorical')
    
    # Create and compile the model
    classifier = Sequential()
    classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
    classifier.add(MaxPooling2D(pool_size=(2, 2)))
    classifier.add(Conv2D(64, (3, 3), activation='relu'))
    classifier.add(MaxPooling2D(pool_size=(2, 2)))
    classifier.add(Flatten())
    classifier.add(Dense(units=128, activation='relu'))
    classifier.add(Dense(units=4, activation='softmax'))
    classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Fit the model
    batches_per_epoch = len(train_set)//steps_per_epoch
    total_batches = batches_per_epoch*epochs
    
    # Generate repeated training data if necessary
    if total_batches > len(train_set):
        print(f"WARNING: Only {len(train_set)} batches available for training. Repeating data...")
        train_generator = train_datagen.flow_from_directory(
            '/content/gdrive/My Drive/dataset_train',
            target_size=(64, 64),
            batch_size=32,
            classes=['category 1', 'category 2', 'category 3', 'category 4'],
            class_mode='categorical'
        )
        train_set = itertools.islice(itertools.cycle(train_generator), total_batches)
    else:
        train_set.reset()

    history = classifier.fit(train_set, steps_per_epoch=batches_per_epoch, epochs=epochs)
    
    # Make a prediction and add to results
    data = []
    results = []
    for f1 in files:
        img = image.load_img(f1, target_size=(64, 64))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        data.append(img)
        result = classifier.predict(img)
        r = np.argmax(result, axis=1)
        results.append(r)

    # Convert results to a 1D numpy array
    y_pred_test = np.array([pred[0] for pred in results])

    # Compute the accuracy score
    test_accuracy = accuracy_score(test_label, y_pred_test)
    
    # Append the results to the list
    model_results.append({'Steps per Epoch': steps_per_epoch, 'Epochs': epochs, 'history': history.history, 
                          'Train Loss': sum(history.history['loss']), 'Train Accuracy': statistics.mean(history.history['accuracy']), 
                          'Test Accuracy': str(test_accuracy*100)+'%'})
    print('*****************************************************************')
    
print('Training complete')


Training model with steps_per_epoch=1 and epochs=1
1/1 [==============================] - 0s 33ms/step
Training model with steps_per_epoch=1 and epochs=2
Epoch 1/2
1/1 [==============================] - 2s 2s/step - loss: 1.3220 - accuracy: 0.3750
Epoch 2/2
1/1 [==============================] - 0s 20ms/step
Training model with steps_per_epoch=1 and epochs=3
Epoch 1/3
1/1 [==============================] - 1s 1s/step - loss: 1.4018 - accuracy: 0.2500
Epoch 2/3
1/1 [==============================] - 1s 551ms/step - loss: 1.7642 - accuracy: 0.2188
Epoch 3/3
1/1 [==============================] - 0s 20ms/step
Training model with steps_per_epoch=2 and epochs=4
Epoch 1/4


2/2 [==============================] - 2s 302ms/step - loss: 1.3079 - accuracy: 0.3571
Epoch 2/4
2/2 [==============================] - 1s 468ms/step - loss: 0.9081 - accuracy: 0.7188
Epoch 3/4
2/2 [==============================] - 1s 519ms/step - loss: 0.4954 - accuracy: 0.9286
Epoch 4/4
1/1 [==============================] - 0s 32ms/step
Training model with steps_per_epoch=2 and epochs=5
Epoch 1/5
2/2 [==============================] - 2s 341ms/step - loss: 1.4630 - accuracy: 0.3036
Epoch 2/5
2/2 [==============================] - 1s 412ms/step - loss: 1.5331 - accuracy: 0.3571
Epoch 3/5
2/2 [==============================] - 1s 458ms/step - loss: 0.8354 - accuracy: 0.6607
Epoch 4/5
2/2 [==============================] - 1s 519ms/step - loss: 0.5885 - accuracy: 0.9107
Epoch 5/5
1/1 [==============================] - 0s 22ms/step
Training model with steps_per_epoch=2 and epochs=6
Epoch 1/6
2/2 [==============================] - 2s 417ms/step - loss: 1.2901 - accuracy: 0.4464
Epoch 2/

1/1 [==============================] - 0s 21ms/step
Training model with steps_per_epoch=5 and epochs=10
Epoch 1/10
3/5 [=================>............] - ETA: 0s - loss: 1.3427 - accuracy: 0.3750

1/1 [==============================] - 0s 21ms/step
Training complete


The total accuracy reported at the end of training is the average of the accuracy calculated at the end of each epoch.
During training, the accuracy at each step is calculated based on the predictions made by the model on the current batch of training data. However, the accuracy reported for each epoch is calculated by aggregating the accuracy across all the batches of data that were used in that epoch.


In [15]:
final_df = pd.DataFrame([{'Steps per Epoch': d['Steps per Epoch'],
                'Epochs': d['Epochs'], 'Train Loss': d['Train Loss'], 
                'Train Accuracy': d['Train Accuracy'] , 'Test Accuracy': d['Test Accuracy'],} for d in model_results])
final_df

,steps_per_epoch,epochs,test_accuracy
0,1,1,37.5%
1,1,2,25.0%
2,1,3,62.5%
3,2,4,87.5%
4,2,5,87.5%
5,2,6,75.0%
6,3,7,87.5%
7,3,8,75.0%
8,5,9,62.5%
9,5,10,37.5%


Repeating data in a dataset can lead to overfitting if the model starts to memorize the data instead of learning the underlying patterns. If the model is repeatedly presented with the same examples during training, it can become overly specialized to those specific examples and perform poorly on new, unseen data.



### Conceptual Questions

<i>4. Discuss the effect of the following on accuracy and loss (train and test): </i>
    
* Increasing the steps_per_epoch
* Increasing the number of epochs

Increasing the steps_per_epoch and the number of epochs can both have an impact on the accuracy and loss of the training and testing process.
<b>When the steps_per_epoch is increased, the model is trained on more batches of data in each epoch.</b> This can lead to a more fine-tuned model with better accuracy on the training data. However, it can also increase the risk of overfitting the model to the training data, as it may not generalize well to new, unseen data.
<b>When the number of epochs is increased, the model is trained on the same data for longer, allowing it to potentially learn more complex patterns and achieve better accuracy on both the training and testing data.</b>

Based on the table, we can observe the effect of changing the hyperparameters steps_per_epoch and epochs on the accuracy of the model.
* For steps_per_epoch = 1, we can see that the accuracy remains constant at 75% for all the different values of epochs. This could be due to the fact that the model is not getting enough data to learn from and hence, not improving its performance.
* When steps_per_epoch is increased to 2 and 3, we see some improvement in the accuracy, with the highest accuracy of 87.5% being achieved when steps_per_epoch = 2 and epochs = 4 and 6 and steps_per_epoch = 3 and epochs = 8.
* Further increasing steps_per_epoch to 5 does not result in a significant improvement in the accuracy, with the highest accuracy of 87.5% being achieved when steps_per_epoch = 5 and epochs = 9.

Overall, increasing steps_per_epoch and epochs can improve the accuracy of the model, but only up to a certain extent. After that, further increasing these hyperparameters may not result in a significant improvement in accuracy and may even lead to overfitting of the model.

<i>5. Name two uses of zero padding in CNN.</i>

<b>Zero padding</b> is a technique used in convolutional neural networks (CNNs) for image processing. 
Two uses of zero padding in CNN are:
* <b>Increasing the size of the output feature maps:</b> Zero padding adds zeros to the border of the input image, which increases the size of the output feature maps. This can be useful when the input size is smaller than the desired output size, as it allows the network to learn more features and capture more information from the image.
* <b>Maintaining spatial resolution:</b> Zero padding can help maintain the spatial resolution of the input image in the output feature maps. When a convolution is applied to an input image, the size of the output feature map can be smaller than the input image, especially when using a small filter size or a large stride. By adding zero padding to the input image, the spatial resolution of the output feature map can be preserved.

<br>

<i>6. What is the use of a 1 x 1 kernel in CNN?</i>
A <b>1x1 convolutional kernel</b> is also called a pointwise convolution, which performs a linear transformation on the input data in the channel dimension only. The 1x1 kernel in a CNN can serve several purposes:
* <b>Dimensionality Reduction:</b> A 1x1 convolution can be used to reduce the number of channels (or features) in the input data. This can help to reduce the computational complexity of the CNN while preserving important features in the data.
* <b>Non-linearity:</b> A 1x1 convolution can introduce non-linearity into the CNN, which can help to improve its performance.
* <b>Feature Combination:</b> A 1x1 convolution can be used to combine features from different channels in the input data. This can help to create more complex representations of the input data, which can improve the performance of the CNN.

<br>

<i>7. What are the advantages of a CNN over a fully connected DNN for this image classification problem?</i>
There are several advantages of a convolutional neural network (CNN) over a fully connected deep neural network (DNN) for image classification:

* <b>Parameter Efficiency:</b> In a CNN, the same filter/kernel is used to scan different parts of the image, thus reducing the number of parameters required compared to a fully connected DNN. This allows the network to be deeper and more efficient in learning complex features.

* <b>Translation Invariance:</b> A CNN can recognize patterns regardless of their position in the image. This is achieved by sharing the same filter/kernel across the entire image. In contrast, a fully connected DNN treats each pixel independently, and therefore is unable to recognize patterns that are shifted or rotated in the image.

* <b>Reduced Overfitting:</b> A CNN is less prone to overfitting than a fully connected DNN. This is because the use of pooling layers in a CNN reduces the dimensionality of the feature maps, thus preventing the network from memorizing the training data.

* <b>Local Receptive Fields:</b> In a CNN, each neuron is only connected to a small region of the input data, which allows the network to learn local features such as edges and textures more efficiently.

Overall, CNNs are more effective at image classification tasks due to their ability to learn and recognize local features in images, their parameter efficiency, and their ability to reduce overfitting.
